In [1]:
import sys
import os

# Add project root to path
project_root = os.path.abspath(os.path.join(os.getcwd(), '../..'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

In [2]:
from src.models.transcription import TranscriptionService
from src.models.diarization import SpeakerDiarizationService
from src.processing.transcript_processor import TranscriptProcessor

# Initialize services
transcription_service = TranscriptionService()
diarization_service = SpeakerDiarizationService()
processor = TranscriptProcessor()

Device set to use cuda


In [3]:
# sample = "../../data/test_data/long_conversation.mp3"
sample = "../../data/test_data/bel.mp4"

In [4]:
sample_transcription = transcription_service.transcribe_with_chunks(sample)
print(len(sample_transcription))
sample_diarization = diarization_service.diarize(sample)
print(len(sample_diarization))
sample_transcript = processor.merge_transcription_with_speakers(sample_transcription, sample_diarization)

# Display results
print(sample_transcript)

/workspaces/audio-analyzer/src/models/transcription.py:88: UserWarning: PySoundFile failed. Trying audioread instead.
  audio_data, sr = librosa.load(audio_path, sr=16000)
/opt/conda/lib/python3.10/site-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)
Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.43.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCac

3


/opt/conda/lib/python3.10/site-packages/pyannote/audio/utils/reproducibility.py:74: ReproducibilityWarning: TensorFloat-32 (TF32) has been disabled as it might lead to reproducibility issues and lower accuracy.
It can be re-enabled by calling
   >>> import torch
   >>> torch.backends.cuda.matmul.allow_tf32 = True
   >>> torch.backends.cudnn.allow_tf32 = True
See https://github.com/pyannote/pyannote-audio/issues/1370 for more details.

  warnings.warn(
/opt/conda/lib/python3.10/site-packages/pyannote/audio/models/blocks/pooling.py:104: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at ../aten/src/ATen/native/ReduceOps.cpp:1823.)
  std = sequences.std(dim=-1, correction=1)


4
[{'timestamp': (0.0, 2.32), 'text': ' Там в Америке меня не хватает.', 'speaker': 'SPEAKER_00'}, {'timestamp': (3.3200000000000003, 11.540000000000001), 'text': ' Вот я бы Трампу прямо в глаза сказал, что нельзя, не что даже, а как нельзя делать.', 'speaker': 'SPEAKER_00'}, {'timestamp': (11.9, 16.14), 'text': ' Вам надо сегодня в США остыть, успокоиться.', 'speaker': 'SPEAKER_00'}]


In [5]:
for some in sample_transcription:
    print(some)

{'text': ' Там в Америке меня не хватает.', 'timestamp': (0.0, 2.32)}
{'text': ' Вот я бы Трампу прямо в глаза сказал, что нельзя, не что даже, а как нельзя делать.', 'timestamp': (3.3200000000000003, 11.540000000000001)}
{'text': ' Вам надо сегодня в США остыть, успокоиться.', 'timestamp': (11.9, 16.14)}


In [6]:
for some in sample_diarization:
    print(some)

{'start': 0.28409375000000003, 'end': 2.32596875, 'speaker': 'SPEAKER_00'}
{'start': 3.27096875, 'end': 5.228468750000001, 'speaker': 'SPEAKER_00'}
{'start': 6.37596875, 'end': 13.98659375, 'speaker': 'SPEAKER_00'}
{'start': 15.31971875, 'end': 16.26471875, 'speaker': 'SPEAKER_00'}


In [7]:
processor.format_transcript_for_display(sample_transcript)

'SPEAKER_00 [0.00s - 2.32s]: Там в Америке меня не хватает.\nSPEAKER_00 [3.32s - 11.54s]: Вот я бы Трампу прямо в глаза сказал, что нельзя, не что даже, а как нельзя делать.\nSPEAKER_00 [11.90s - 16.14s]: Вам надо сегодня в США остыть, успокоиться.'

In [8]:
from src.clustering.semantic_cluster import SemanticCluster

cluster_service = SemanticCluster()

clusters = cluster_service.fit_transform(
    sample_transcript
)

2025-03-09 10:01:35,859 - BERTopic - Embedding - Transforming documents to embeddings.


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2025-03-09 10:01:38,112 - BERTopic - Embedding - Completed ✓
2025-03-09 10:01:38,112 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm


Notice: Short transcript processed as single topic. Cannot use scipy.linalg.eigh for sparse A with k >= N. Use scipy.linalg.eigh(A.toarray()) or reduce k.


/opt/conda/lib/python3.10/site-packages/umap/spectral.py:519: RuntimeWarning:

k >= N for N * N square matrix. Attempting to use scipy.linalg.eigh instead.

/opt/conda/lib/python3.10/site-packages/umap/spectral.py:519: RuntimeWarning:

k >= N for N * N square matrix. Attempting to use scipy.linalg.eigh instead.



In [9]:
len(clusters)

1

In [10]:
for some in clusters:
    print(some)

[{'timestamp': (0.0, 2.32), 'text': ' Там в Америке меня не хватает.', 'speaker': 'SPEAKER_00'}, {'timestamp': (3.3200000000000003, 11.540000000000001), 'text': ' Вот я бы Трампу прямо в глаза сказал, что нельзя, не что даже, а как нельзя делать.', 'speaker': 'SPEAKER_00'}, {'timestamp': (11.9, 16.14), 'text': ' Вам надо сегодня в США остыть, успокоиться.', 'speaker': 'SPEAKER_00'}]


In [11]:
formatted_clusters = processor.format_transcript_by_topic(clusters)
formatted_clusters

['SPEAKER_00 [00:00:00.00-00:00:02.32]: Там в Америке меня не хватает.\nSPEAKER_00 [00:00:03.32-00:00:11.54]: Вот я бы Трампу прямо в глаза сказал, что нельзя, не что даже, а как нельзя делать.\nSPEAKER_00 [00:00:11.90-00:00:16.14]: Вам надо сегодня в США остыть, успокоиться.']

In [12]:
from src.models.summarization import SummarizationService

summarization_service = SummarizationService()
summaries = summarization_service.summarize_all_topics(formatted_clusters)

In [13]:
for summary in summaries:
    print(summary)

## Summary
The speaker expresses concern about a situation in America, suggesting they would directly confront President Trump on certain actions that are unacceptable. They also advise the person to calm down and cool off in the United States.

## Notable Moments
- 00:02:32 Quote: "Там в Америке меня не хватает." - [Brief context of speaker's concern about America]
- 00:11:54 Quote: "Вынад сегодня в США остыть, успокоиться." - [Speaker advising to calm down and cool off in the US]
